In [1]:
# Prepare the chat-completion version of the agent trainnig with ReFT
from data.corpus import mock_conversation
mock_conversation # list of string

['Agent: Okay, you are okay with the recording?',
 'Customer: Yeah, sure.',
 'Agent: Okay, so this is our first meeting?',
 'Customer: Yes.',
 'Agent: Hi, my name is Marge. You are?',
 'Customer: Alice. Alice is my calling card. Thank you.',
 'Agent: Thank you for your time today. I really appreciate it. How was your trip?',
 'Customer: It was okay. Yeah, I was just nearby. I was having coffee with a friend.',
 'Agent: Is it your first time to be in our BGC office?',
 "Customer: Yeah, it's my first time talking about insurance as well because I've never bought insurance before. But a lot of my friends are getting it, and it sounds like it's important.",
 "Agent: Okay, and I'm very thankful to my friend Berna because she recommended me to you. So, yeah, thank you for your time. I know you just came here from your lunch break, so I don't want to hold you too long. If you have any questions, feel free to ask me.",
 'Customer: Okay.',
 "Agent: By the way, I'm just going to show you a Power

In [16]:
# Prepare dataset --> prompt & completion
# Address the conversation history in a specific way | put that into the system prompt should be fine already 
import datasets

def prepare_sale_dataset():
    from data.corpus import mock_conversation

    dataset_dict = {
        "system_prompt": [],
        "prompt": [],
        "completion": []
    }

    for i_round in range(len(mock_conversation)//2):
        mock_history = mock_conversation[:2*i_round + 1]
        customer_utterance = mock_conversation[2*i_round + 1]
        sale_response = mock_conversation[2*i_round + 2]

        system_prompt = "[Conversation History] " + " ".join(mock_history) + "[End]" 
        prompt = "[Customer Utterance] " + customer_utterance + " [Sale Response] "
        completion = sale_response

        dataset_dict["system_prompt"].append(system_prompt)
        dataset_dict["prompt"].append(prompt)
        dataset_dict["completion"].append(completion)

    # prepare a huggingface dataset object
    dataset = datasets.Dataset.from_dict(dataset_dict)

    return dataset

dataset = prepare_sale_dataset()

import transformers 

model_name_or_path = "meta-llama/Meta-Llama-3-8B-Instruct"
model_max_length = 2048
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name_or_path, model_max_length=model_max_length, 
    padding_side="right", use_fast=False)

system_prompt = "Follow the instruction closely and provide your answer."
query_list = [tokenizer.apply_chat_template(
        [
            {"role": "system", "content": data["system_prompt"]}, 
            {"role": "user", "content": data['prompt']}
        ], tokenize=False
) for data in dataset]
answer_list = [
        tokenizer.apply_chat_template(
            [{"role": "assistant", "content": data['completion']}], tokenize=False,
        )[len(tokenizer.bos_token):] for data in dataset]



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
